In [0]:
# Install compatible versions of numpy and pandas
# %pip install numpy==1.23.5 pandas==1.5.3

from pyspark.sql import SparkSession
from databricks.vector_search.client import VectorSearchClient
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings

# Initialize Spark session
# spark = SparkSession.builder.getOrCreate()

vs_endpoint_name = "community_bricks_endpoint"
vs_index_full_name = "access_assist.curated.airbnb_exploded_vc_idex"

embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

def get_retriever(persist_dir: str = None):
    vsc = VectorSearchClient()
    vs_index = vsc.get_index(vs_endpoint_name, vs_index_full_name)
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="exploded_chunk", embedding=embedding_model
    )
    return vectorstore.as_retriever(search_kwargs={"k": 10})

vectorstore = get_retriever()
similar_documents = vectorstore.invoke("""
                                       I am looking for an accessible hotel for persons with a disability: the location preferably will have wheelchair access, an elevator, and rooms equipped with grab bars in the bathroom. It should also be close to public transportation and have accessible parking.
                                       """)
print(f"Relevant documents: {similar_documents}")

In [0]:
from langchain.chat_models import ChatDatabricks

chat_model = ChatDatabricks(endpoint="databricks-llama-4-maverick", max_tokens=300)
print(f"Test chat model: {chat_model.invoke('What is Generative AI?')}")

In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatDatabricks

TEMPLATE = """You are an assistant trying to find accessible places to stay. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know

<context>
{context}
</context>

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["context", "question"]
)

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [0]:
question = {"query": "Find me an accessible hotel via wheelchair"}
answer = chain.invoke(question)
print(answer)